<a href="https://colab.research.google.com/github/EishaMazhar/Glassdoor-Salary-Prediction/blob/main/Model%20Building-Job%20Description-NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
# train test split 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("preprocessed_data.csv")
df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,hourly,employer_provided,min_salary,max_salary,avg_salary,job_state,age,job_title_tag,seniority,desc_len,python,R,spark,aws,excel,sql,django,hadoop,java,db,tableau,azure
0,0,Data Scientist,$74K - $139K,Intro (Use Font Arial 12):\r\nAs a Data Scient...,4.4,Applied Information Sciences,"Chevy Chase, MD",501 to 1000 Employees,1982,Unknown,IT Services,Information Technology,$50 to $100 million (USD),0,0,74,139,106.5,MD,39,data scientist,mid/other,755,0,0,0,0,0,1,0,0,0,1,0,1
1,1,Web Developer,$41K - $83K,We are the fastest growing employer of emergin...,4.0,Revature,"Tampa, FL",1001 to 5000 Employees,2003,Unknown,IT Services,Information Technology,$100 to $500 million (USD),0,0,41,83,62.0,FL,18,other,mid/other,625,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Data Analyst,$43K - $88K,"HR Data Analyst - Direct Hire - Buffalo, NY\r\...",4.1,Systems Personnel,"Niagara Falls, NY",1 to 50 Employees,1996,Unknown,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),0,0,43,88,65.5,NY,25,analyst,mid/other,1366,0,0,0,0,0,1,0,0,0,0,1,0
3,3,Database Administrator,$68K - $136K,"Secure our Nation, Ignite your Future\r\nBecom...",4.2,ManTech International Corporation,"Fort Meade, MD",5001 to 10000 Employees,1968,Unknown,Research & Development,Business Services,$1 to $2 billion (USD),0,0,68,136,102.0,MD,53,other,mid/other,765,0,0,0,0,0,0,0,0,0,1,0,0
4,4,iOS Developer,$72K - $104K,Are you an iOS developer who loves to create m...,4.1,Garmin,"Olathe, KS",10000+ Employees,1989,Unknown,Consumer Products Manufacturing,Manufacturing,$2 to $5 billion (USD),0,0,72,104,88.0,KS,32,other,mid/other,929,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4912,4912,Front End Developer,$58K - $117K,US Citizenship is required\r\nFront end Develo...,-1.0,EDR Technology,"Ashburn, VA",Unknown,-1,Unknown,-1,-1,Unknown / Non-Applicable,0,0,58,117,87.5,VA,-1,other,mid/other,1314,0,0,0,0,0,0,0,0,1,1,0,0
4913,4913,Sr Software Quality Engineer,$78K - $147K,PRA Health Sciences is seeking the best and br...,4.0,PRA Health Sciences,"San Diego, CA",10000+ Employees,1976,Unknown,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$2 to $5 billion (USD),0,0,78,147,112.5,CA,45,software developer/engineer,senior,980,0,0,0,0,0,0,0,0,0,0,0,0
4914,4914,Lead Mobile Engineer,$79K - $162K,About Us\r\nWe are a fintech startup with a mi...,-1.0,Treecard,"Austin, TX",Unknown,-1,Unknown,-1,-1,Unknown / Non-Applicable,0,0,79,162,120.5,TX,-1,other,senior,812,0,0,0,0,0,0,0,0,0,0,0,0
4915,4915,Cloud Infrastructure Engineer (SRE/Java),$61K - $132K,"Every day, Global Payments makes it possible f...",3.5,Global Payments,"Columbus, GA",10000+ Employees,1967,Unknown,Financial Transaction Processing,Finance,$5 to $10 billion (USD),0,0,61,132,96.5,GA,54,other,senior,641,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'hourly',
       'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'job_state', 'age', 'job_title_tag', 'seniority', 'desc_len', 'python',
       'R', 'spark', 'aws', 'excel', 'sql', 'django', 'hadoop', 'java', 'db',
       'tableau', 'azure'],
      dtype='object')

In [4]:
df['Job Title'].unique()

array(['Data Scientist', 'Web Developer', 'Data Analyst', ...,
       'Lead Mobile Engineer', 'Cloud Infrastructure Engineer (SRE/Java)',
       'Development Chapter Lead in Retail'], dtype=object)

In [5]:
df_model = df[['Job Title', 'Rating','Company Name', 'Company Name', 'Size',
               'Type of ownership', 'Industry', 'Sector', 'Revenue', 'hourly', 'employer_provided',
               'avg_salary', 'job_state', 'age',
               'job_title_tag', 'seniority', 'desc_len', 'python', 'R', 'spark', 'aws',
               'excel', 'sql', 'django', 'hadoop', 'java', 'db', 'tableau', 'azure']]
               

In [6]:
dummy_df = pd.get_dummies(df_model)

X = dummy_df.drop('avg_salary', axis =1)
y = dummy_df.avg_salary.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [7]:
df['Job Description'].map(lambda x: len(x)).mean()

1031.6672767947937

In [8]:
! pip install nltk

In [9]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [10]:
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer("[a-z']+")

In [11]:
# preprocess text
def get_tokens(description):
    text_tokens = tokenizer.tokenize(description)
    stemmed_tokens = [stemmer.stem(x) for x in text_tokens]
    return stemmed_tokens

In [12]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,1), tokenizer=get_tokens)

In [13]:
transformed_text = vectorizer.fit_transform(df['Job Description'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

In [14]:
transformed_text

<4917x12791 sparse matrix of type '<class 'numpy.float64'>'
	with 348052 stored elements in Compressed Sparse Row format>

In [15]:
X_train, X_test, y_train, y_test = train_test_split(transformed_text, df['avg_salary'], test_size=0.3)

In [16]:
from sklearn.naive_bayes import GaussianNB

In [17]:
gnb_model = GaussianNB().fit(X_train.toarray(), y_train.astype('int'))

In [18]:
y_pred = gnb_model.predict(X_test.toarray())

In [19]:
mean_squared_error(y_test, y_pred)

623.5960365853658

In [20]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor().fit(X_train.toarray(), y_train.astype('int'))

In [ ]:
y_pred = rf_model.predict(X_test.toarray())

In [ ]:
mean_squared_error(y_test, y_pred)

In [22]:

!pip install tensorflow_hub

In [23]:
import tensorflow as tf
import tensorflow_hub as hub

In [24]:

model_link = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model_link, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [25]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 128)               2688      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 411,029
Trainable params: 411,029
Non-trainable params: 0
__________________________________________________

In [26]:
model.compile(optimizer='adam',
              loss=tf.losses.MeanSquaredError(),
              metrics=['mae', 'mse', 'msle'])

In [27]:
history = model.fit(df['Job Description'], df['avg_salary'],
                    epochs=50,
                    batch_size=256,
                    verbose=1)

Epoch 1/50
20/20 [==============================] - 1s 22ms/step - loss: 7240.6523 - mae: 80.3475 - mse: 7240.6523 - msle: 8.1974
Epoch 2/50
20/20 [==============================] - 0s 24ms/step - loss: 3702.2881 - mae: 53.1668 - mse: 3702.2881 - msle: 1.3346
Epoch 3/50
20/20 [==============================] - 0s 24ms/step - loss: 1555.5347 - mae: 29.6674 - mse: 1555.5347 - msle: 0.2423
Epoch 4/50
20/20 [==============================] - 0s 22ms/step - loss: 1374.3898 - mae: 28.0389 - mse: 1374.3898 - msle: 0.2300
Epoch 5/50
20/20 [==============================] - 0s 21ms/step - loss: 1277.0636 - mae: 26.7696 - mse: 1277.0636 - msle: 0.1914
Epoch 6/50
20/20 [==============================] - 0s 22ms/step - loss: 1197.3607 - mae: 25.8667 - mse: 1197.3607 - msle: 0.1743
Epoch 7/50
20/20 [==============================] - 0s 22ms/step - loss: 1102.7515 - mae: 24.7947 - mse: 1102.7515 - msle: 0.1546
Epoch 8/50
20/20 [==============================] - 0s 23ms/step - loss: 1028.2662 - mae: 

In [28]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [29]:
model.predict(["""As a Data Scientist Architect you will be applying new and innovative methods to address some of our most important intelligence problems \
What You'll Be Doing:\
Knowledge and experience in designing and implementing AI apps and agents that use the Microsoft Azure Cognitive Services, Azure Bot Service, Azure Cognitive Search, and data storage options \
Analyzing requirements for AI solutions Experience with NoSQL databases Experience with applying Data Science techniques to overhead imagery (computer vision, object detection \
Ability to work autonomously and collaboratively as part of a team to both teach and learn every day"""])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


array([[111.76984]], dtype=float32)

In [30]:
model.predict(["""We are looking for a full time iOS Software Engineer to join our team in Olathe, KS. In this role, you’ll be responsible for architecting and developing complex software utilized by the suite of Garmin mobile applications. Responsibilities include:
Develop software using Swift and Objective-C
Utilize tools in the software development process such as Git, Gerrit, Bitbucket, Jira, Confluence, Xcode, and Jenkins
Create and implement designs for software components within a larger project"""])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


array([[95.61134]], dtype=float32)